In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Reshape, Concatenate, LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#Wrapper class for model
#Image dimensions standardisation
image_dimensions={'height':256,'width':256,'channels':3}

class Classifier:
    def __init__():
        self.model = 0

    def predict(self, x):
        if x.size == 0:
            return []
        return self.model.predict(x)

    def fit(self, x, y):
        return self.model.train_on_batch(x, y)

    def get_accuracy(self, x, y):
        return self.model.test_on_batch(x, y)

    def load(self, path):
        self.model.load_weights(path)

class Meso4(Classifier):
  #Initiialising the class
    def __init__(self, learning_rate = 0.001):
        self.model = self.init_model()
        optimizer = Adam(learning_rate = learning_rate)
        self.model.compile(optimizer = optimizer, loss = 'mean_squared_error', metrics = ['accuracy'])

    def init_model(self):
      #Input Layer
        x = Input(shape = (image_dimensions['height'], image_dimensions['width'], image_dimensions['channels']))

#4 convolutional blocks
        x1 = Conv2D(8, (3, 3), padding='same', activation = 'relu')(x)
        x1 = BatchNormalization()(x1)
        x1 = MaxPooling2D(pool_size=(2, 2), padding='same')(x1)

        x2 = Conv2D(8, (5, 5), padding='same', activation = 'relu')(x1)
        x2 = BatchNormalization()(x2)
        x2 = MaxPooling2D(pool_size=(2, 2), padding='same')(x2)

        x3 = Conv2D(16, (5, 5), padding='same', activation = 'relu')(x2)
        x3 = BatchNormalization()(x3)
        x3 = MaxPooling2D(pool_size=(2, 2), padding='same')(x3)

        x4 = Conv2D(16, (5, 5), padding='same', activation = 'relu')(x3)
        x4 = BatchNormalization()(x4)
        x4 = MaxPooling2D(pool_size=(4, 4), padding='same')(x4)

        y = Flatten()(x4)
        y = Dropout(0.5)(y)
        y = Dense(16)(y)
        y = LeakyReLU(alpha=0.1)(y)
        y = Dropout(0.5)(y)
        y = Dense(1, activation = 'sigmoid')(y)

        return Model(inputs = x, outputs = y)

#Initializing model and loading weights
model=Meso4()
model.load("../model_weights/Meso4_DF (1).h5")
import cv2
import numpy as np
from typing import List

# Function to preprocess the image
def preprocess_image(image, target_size=(256, 256)):
    # Convert BGR to RGB
    img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Resize the image to match model input
    img = cv2.resize(img, target_size)

    # Normalize the image
    img = img.astype('float32') / 255.0

    # Expand dimensions to match model input shape
    img = np.expand_dims(img, axis=0)  # Add batch dimension

    return img

# Function to analyze the video and make predictions
def analyze_video(video_path, model):
    sum_predictions = 0
    list_of_predictions = []

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Preprocess the frame
        preprocessed_frame = preprocess_image(frame)

        # Make predictions
        predictions = model.predict(preprocessed_frame)

        # Display predictions on the frame
        prediction_value = float(predictions[0][0])
        list_of_predictions.append(prediction_value)


        # Display the frame
        # cv2.imshow('Video', frame)  # Uncomment to display video with prediction

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release resources
    cap.release()
    cv2.destroyAllWindows()

    # Calculate the average of the predictions
    for i in list_of_predictions:
        sum_predictions+=i
   # Check if any predictions were made
    if len(list_of_predictions) > 0:
        average_prediction = sum_predictions / len(list_of_predictions)
    else:
        average_prediction = 0  # or handle this case as you prefer
    

    # Determine if the video is real or fake based on the average prediction
    return average_prediction,list(list_of_predictions)





In [ ]:
from fastapi import FastAPI, File, UploadFile
import os
from fastapi.responses import HTMLResponse, JSONResponse
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
import nest_asyncio
import cv2
import numpy as np


nest_asyncio.apply()

app = FastAPI()
UPLOAD_DIRECTORY = "./uploaded_files"

# Ensure the directory exists
os.makedirs(UPLOAD_DIRECTORY, exist_ok=True)

app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://localhost:3001"],  # React dev server URL
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.post("/upload")
async def upload_file(video: UploadFile = File(None), audio: UploadFile = File(None)):
    result = None
    random_array = []
    micro=None
 
   
    
    if video:
        video_path = os.path.join(UPLOAD_DIRECTORY, video.filename)
        with open(video_path, "wb") as buffer:
            buffer.write(await video.read())

        # Call the video analysis function
     
        result, random_array = analyze_video(video_path,model)
        if result>0.5:
            micro="The Microexpressions are human-like and this is not a deepfake."
        else:
            micro="The Microexpressions are not human-like and this is a deepfake."
            
 

    return {"result":result,"micro": micro, "random_array": random_array}

# Run the FastAPI app
if __name__ == "__main__":
    uvicorn.run(app, host="127.0.0.1", port=8000)




INFO:     Started server process [56013]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


1/1 [==============================] - ETA: 0s

2024-09-21 12:32:57.209822: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1/1 [==============================] - 0s 9ms/step
INFO:     127.0.0.1:64714 - "POST /upload HTTP/1.1" 200 OK
1/1 [==============================] - 0s 10ms/step
INFO:     127.0.0.1:64967 - "POST /upload HTTP/1.1" 200 OK
